# 1. Group by and Order by

我们在上一篇中介绍了 [Where 子句](https://blog.csdn.net/weixin_45488228/article/details/104377915)，接下来我们将使用 Group by 和 Order by 子句，对数据进行聚合和排序。

- 使用Jupyter Notebook 运行 SQL 语句需安装 ipython-sql

- %sql 以及 %%sql 为在 Notebook 中运行 SQL 语句，在 SQLite 命令行或 SQLite Stiduo 中不需要 %sql 或 %%sql

载入 SQL 以及连接 SQLite：

In [1]:
%load_ext sql
%sql sqlite:///DataBase/weather_stations.db

'Connected: @DataBase/weather_stations.db'

本文将使用 [weather_stations.db](https://github.com/X1AOX1A/SQL_Notes/blob/master/SQLite/DataBase/weather_stations.db) 数据库，其中包含了 STATION_DATA 表。

首先查看 STATION_DATA 表中的数据：

In [3]:
%sql select * from station_data limit 0,5; -- 筛选前五行

 * sqlite:///DataBase/weather_stations.db
Done.


station_number,report_code,year,month,day,dew_point,station_pressure,visibility,wind_speed,temperature,precipitation,snow_depth,fog,rain,hail,thunder,tornado
143080,34DDA7,2002,12,21,33.8,987.4,3.4,0.2,36,0,None,1,1,1,1,1
766440,39537B,1998,10,1,72.7,1014.6,5.9,6.7,83.3,0,None,0,0,0,0,0
176010,C3C6D5,2001,5,18,55.7,None,7.3,4.3,69.1,0,None,0,0,0,0,0
125600,145150,2007,10,14,33,None,6.9,2.5,39.7,0,None,0,0,0,0,0
470160,EF616A,1967,7,29,65.6,None,9.2,1.2,72.4,0.04,None,0,0,0,0,0


## 1.1 Group Records

首先从最简单的聚合方法开始：计数：

In [4]:
%%sql
select count(*) as record_cound from station_data;

 * sqlite:///DataBase/weather_stations.db
Done.


record_cound
28000


**count(*)** 意味着计算记录的长度，你也可以和其他 SQL 操作符结合起来使用，比如 **where**，我们可以这样计算 tornado 出现的次数：

In [6]:
%%sql
select count(*) as record_count from station_data
where tornado == 1;

 * sqlite:///DataBase/weather_stations.db
Done.


record_count
3000


我们找到了 3000 条包含 tornado 的记录，但如果我们想要按年计数呢？我们可以这样写：

In [11]:
%%sql
select year, 
count(*) as record_count 
from station_data
where tornado == 1
group by year
limit 0,3; -- 只展示前三条

 * sqlite:///DataBase/weather_stations.db
Done.


year,record_count
1937,3
1941,3
1942,3


我们现在可以看到每年的计数，让我们拆分下这个查询来看看怎么执行的：

```  SQL
select year,              -- 1. 首先，我们选择了 year（select year）
count(*) as record_count  -- 2. 然后我们用 **count(\*)** 对筛选的记录进行了计数
from station_data
where tornado == 1        -- 3. 我们筛选了 tornado 为 true 的数据
group by year             -- 4. 最后，按年进行分类
```

我们也可以在多个 field 上进行聚合：

In [12]:
%%sql
select year, month,
count(*) as record_count
from station_data
where tornado == 1
group by year
limit 0,3;

 * sqlite:///DataBase/weather_stations.db
Done.


year,month,record_count
1937,7,3
1941,8,3
1942,10,3


In [ ]:
此外，在使用 ***group by* 时，我们可以用 **序数位置**（） 而不是列名

# 参考资料

[1] [Thomas Nield.Getting Started with SQL](https://github.com/X1AOX1A/SQL_Notes/blob/master/getting_started_with_sql/Getting%20Started%20with%20SQL.pdf)[M].US: O’Reilly, 2016: 29-37

